<h2>Draftkings data was the only data considered</h2>

<h3>Extract unique Players and count how many times they appear</h3>

In [0]:
file_location = "/FileStore/tables/NHL_2019_2020_DFS_Dataset-1.csv"
file_type = "csv"

initialData = sc.textFile(file_location)
#remove entries that have an #N/A value and cache for future use
cleanData = initialData.filter(lambda x: x.split(",")[9] != "#N/A" and x.split(",")[10] != "#N/A" and x.split(",")[11] != "#N/A" and x.split(",")[12] != "#N/A")
cleanData.cache()

#unique players and how many times they are in the table
uniquePlayers = cleanData.map(lambda x: [int(x.split(",")[3]), 1]).reduceByKey(lambda a, b: a + b)

<h3>Calculate players total score and average score</h3>
<h4>find superstar players (top 10%)</h4>

In [0]:
playerTotalScore = cleanData.map(lambda x: [int(x.split(",")[3]), float(x.split(",")[13])]).reduceByKey(lambda a, b: a + b)

games = cleanData.map(lambda x: [x.split(",")[1], x.split(",")[2]]).reduceByKey(lambda a, b: a)
print(games.count())

# join playerTotalScore and uniquePlayers results in a tupple like this:
# (playerID, [total score, records in table])
playerAverage = playerTotalScore.join(uniquePlayers).map(lambda x: [x[0], x[1][0]/x[1][1]])

centersAverage = cleanData.filter(lambda x: x.split(",")[9] == "C").map(lambda x: [int(x.split(",")[3]), x.split(",")[9]]).reduceByKey(lambda a, b: a).join(playerAverage)
wingmanAverage = cleanData.filter(lambda x: x.split(",")[9] == "W").map(lambda x: [int(x.split(",")[3]), x.split(",")[9]]).reduceByKey(lambda a, b: a).join(playerAverage)
defensemenAverage = cleanData.filter(lambda x: x.split(",")[9] == "D").map(lambda x: [int(x.split(",")[3]), x.split(",")[9]]).reduceByKey(lambda a, b: a).join(playerAverage)
goaliesAverage = cleanData.filter(lambda x: x.split(",")[9] == "G").map(lambda x: [int(x.split(",")[3]), x.split(",")[9]]).reduceByKey(lambda a, b: a).join(playerAverage)

nC = int(centersAverage.count()/10)
nW = int(wingmanAverage.count()/10)
nD = int(defensemenAverage.count()/10)
nG = int(goaliesAverage.count()/10)

superStarCenters = centersAverage.takeOrdered(nC, key = lambda x: -x[1][1])
superStarWingmen = wingmanAverage.takeOrdered(nW, key = lambda x: -x[1][1])
superStarDefensemen = defensemenAverage.takeOrdered(nD, key = lambda x: -x[1][1])
superStarGoalies = goaliesAverage.takeOrdered(nG, key = lambda x: -x[1][1])
print("Centers\n")
print(superStarCenters)
print("\nWingmen\n")
print(superStarWingmen)
print("\nDefensemen\n")
print(superStarDefensemen)
print("\nGoalies\n")
print(superStarGoalies)

1146
Cetners

[(3041969, ('C', 18.049382716049383)), (2562637, ('C', 17.472727272727273)), (4024123, ('C', 17.467567567567567)), (3895074, ('C', 16.29090909090909)), (3114727, ('C', 15.767123287671232)), (3648002, ('C', 14.847692307692311)), (3124, ('C', 14.45925925925926)), (5037, ('C', 14.019642857142859)), (3042083, ('C', 13.195238095238096)), (3904173, ('C', 13.114285714285716)), (2280, ('C', 13.108823529411769)), (5160, ('C', 12.595522388059702)), (3114, ('C', 12.500000000000004)), (2562624, ('C', 12.034328358208958)), (4233566, ('C', 11.679518072289161)), (2562632, ('C', 11.57916666666667)), (3042002, ('C', 11.314285714285715)), (2590852, ('C', 11.311904761904765)), (3041970, ('C', 11.215384615384616)), (3114755, ('C', 11.184057971014497)), (5430, ('C', 11.102564102564104)), (3151187, ('C', 10.927631578947368)), (3669, ('C', 10.753246753246756)), (5349, ('C', 10.744444444444445)), (5798, ('C', 10.70769230769231)), (3961, ('C', 10.605333333333334)), (3899946, ('C', 10.479487179487181)), (3041994, ('C', 10.149350649350652)), (2562601, ('C', 10.105000000000002))]

Wingmen

[(3101, ('W', 17.171232876712327)), (3114778, ('W', 17.08783783783784)), (3114727, ('W', 15.767123287671232)), (3735, ('W', 15.62207792207792)), (4005, ('W', 15.02625)), (3891952, ('W', 14.700000000000001)), (5037, ('W', 14.019642857142859)), (2563060, ('W', 13.892307692307691)), (3899952, ('W', 13.334666666666669)), (3042083, ('W', 13.195238095238096)), (2591155, ('W', 12.881481481481485)), (4024820, ('W', 12.682608695652176)), (3042037, ('W', 12.651219512195123)), (3899938, ('W', 12.54727272727273)), (3852, ('W', 12.423684210526316)), (5614, ('W', 12.326865671641793)), (3899937, ('W', 12.119047619047622)), (2562624, ('W', 12.034328358208958)), (5545, ('W', 11.850000000000003)), (2968772, ('W', 11.618750000000002)), (5251, ('W', 11.585937500000002)), (4319858, ('W', 11.54057971014493)), (5103, ('W', 11.494117647058825)), (5428, ('W', 11.469444444444447)), (2590852, ('W', 11.311904761904765)), (2562606, ('W', 11.200000000000003)), (5430, ('W', 11.102564102564104)), (2562609, ('W', 10.96818181818182)), (3114743, ('W', 10.9484375)), (4352683, ('W', 10.947058823529416)), (2967072, ('W', 10.916250000000002)), (3114736, ('W', 10.895833333333336)), (3114741, ('W', 10.81066666666667)), (3042095, ('W', 10.751724137931037)), (5349, ('W', 10.744444444444445)), (2592095, ('W', 10.67857142857143)), (2501107, ('W', 10.660784313725493)), (4024854, ('W', 10.653424657534249)), (3961, ('W', 10.605333333333334)), (5793, ('W', 10.574358974358976)), (2563039, ('W', 10.531168831168833)), (3899978, ('W', 10.430000000000001)), (3699, ('W', 10.201960784313727)), (5833, ('W', 10.16065573770492)), (3041994, ('W', 10.149350649350652)), (3042086, ('W', 10.118750000000002)), (2976844, ('W', 10.036170212765962)), (5550, ('W', 9.965753424657537)), (3792, ('W', 9.90769230769231)), (5835, ('W', 9.90769230769231))]

Defensemen

[(5436, ('D', 15.28714285714286)), (2562605, ('D', 14.193750000000003)), (5118, ('D', 13.71830985915493)), (5169, ('D', 12.516000000000002)), (3042055, ('D', 12.25)), (2562608, ('D', 12.2234375)), (2300, ('D', 12.175714285714287)), (4013, ('D', 12.081818181818184)), (3539, ('D', 11.958333333333337)), (3464, ('D', 11.867567567567567)), (5157, ('D', 11.609210526315792)), (2278, ('D', 11.585135135135138)), (3006, ('D', 11.323880597014924)), (5131, ('D', 11.200000000000001)), (3899972, ('D', 11.140845070422538)), (3041992, ('D', 10.784615384615387)), (4233563, ('D', 10.68714285714286)), (3042019, ('D', 10.566666666666668)), (2976833, ('D', 10.294117647058826)), (3041997, ('D', 10.24794520547945)), (3069341, ('D', 10.193055555555556)), (2976839, ('D', 9.919117647058826)), (5164, ('D', 9.834545454545456)), (4233536, ('D', 9.629487179487183)), (2952729, ('D', 9.582089552238807)), (3042122, ('D', 9.510975609756098)), (5079, ('D', 9.456451612903225)), (3024798, ('D', 9.41095890410959)), (3114769, ('D', 9.400000000000002))]

Goalies

[(3151297, ('G', 19.876923076923074)), (4

<h3>Group players by role and calculate average salary of role</h3>

In [0]:
roleTotalSalaryRDD = cleanData.map(lambda x: [x.split(",")[9], int(x.split(",")[11])]).reduceByKey(lambda a, b: a + b)
playersInEachRole = cleanData.map(lambda x: [x.split(",")[9], 1]).reduceByKey(lambda a, b: a + b)
averageRollSalery = roleTotalSalaryRDD.join(playersInEachRole).map(lambda x: [x[0], x[1][0]/x[1][1]])
print(averageRollSalery.collect())

[['C', 4183.592644978784], ['W', 3933.120282109207], ['D', 3638.8981515041683], ['G', 7622.584856396867]]